# Préparation et nettoyage des données DVF

**Objectif** : préparer un jeu de données propre pour l'analyse des **locaux commerciaux / boutiques** à **Paris** (arrondissements, prix/m², année, géolocalisation).



In [30]:
import pandas as pd
import numpy as np
import os

In [45]:
df = pd.read_csv('dvf.csv')

C:\Users\victo\AppData\Local\Temp\ipykernel_13904\2023539279.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dvf.csv')


##### on filtre les colonnes importantes

In [46]:
colonnes_importantes = [
    'date_mutation',
    'valeur_fonciere',
    'code_postal',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'longitude',
    'latitude',
    'adresse_nom_voie',
    'nom_commune'
]

df = df[colonnes_importantes]

##### On enlève les lignes qui contiennent des valeurs NaN

In [47]:
df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 
                       'nombre_pieces_principales',
                       'latitude', 'longitude'])

In [48]:
df.isna().sum()

date_mutation                 0
valeur_fonciere               0
code_postal                  79
type_local                    0
surface_reelle_bati           0
nombre_pieces_principales     0
longitude                     0
latitude                      0
adresse_nom_voie              4
nom_commune                   0
dtype: int64

##### On modifie le typage data en datetime

In [49]:
df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')

##### On ne retient que le type : "Local industriel. commercial ou assimilé"

In [50]:
type_autorise = ['Local industriel. commercial ou assimilé']
mask_type = df['type_local'].isin(type_autorise)
df = df[mask_type]

##### Puis on retire du dataframe la colonne "type_local"

In [51]:
df = df.drop('type_local', axis=1)

##### On s'intéresse uniquement au département de Paris

In [52]:
mask_paris = df['code_postal'].astype(str).str.startswith('75')
df = df[mask_paris]

##### Création d'une feature prix m²

In [53]:
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

##### On supprime les valeurs aberrantes au m²

In [54]:
df = df[(df['prix_m2'] > 500) & (df['prix_m2'] < 50000)]

##### Extraction de l'arrondissement (normalisation de code_postal)

In [55]:
df['code_postal'] = df['code_postal'].astype(int).astype(str).str.zfill(5)
df['arrondissement'] = pd.to_numeric(df['code_postal'].str[-2:], errors='coerce')
df.loc[~df['arrondissement'].between(1,20), 'arrondissement'] = np.nan
df['arrondissement'] = df['arrondissement'].dropna().astype(int)

##### on supprime la colonne code_postal (c'est toujours Paris)

In [56]:
df = df.drop('code_postal', axis=1)

##### Certains nom de communes ne sont pas Paris. On les supprime

In [59]:
df = df[df['nom_commune'].str.startswith('Paris')]

enregistrement du fichier csv clean

In [61]:
df.to_csv("dvf_nettoye.csv", sep=';', index=False, encoding='utf-8')

# Nettoyage BDD Loyer

In [ ]:
df = pd.read_csv("loyers.csv", sep=';', encoding='utf-8')

##### On supprime les colonnes inutiles

In [ ]:
colonnes_a_supprimer = ["Secteurs géographiques", "Numéro du quartier", "Ville","geo_shape"]
df = df.drop(columns=colonnes_a_supprimer, errors='ignore')

##### Le dataframe ne contient pas de valeurs NaN ou null, ni de doublons

##### Nettoyage des coordonnées GPS (format "lat, lon")

In [ ]:
df[['lat', 'lon']] = df['geo_point_2d'].str.split(',', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

suppression de la colonne 'geo_point_2d'

In [ ]:
df = df.drop('geo_point_2d', axis=1)

In [ ]:
df.head()

,Année,Nom du quartier,Nombre de pièces principales,Epoque de construction,Type de location,Loyers de référence,Loyers de référence majorés,Loyers de référence minorés,Numéro INSEE du quartier,lat,lon
0,2024,Porte-Saint-Martin,3,Apres 1990,meublé,25.5,30.6,17.9,7511039,48.871245,2.361504
1,2024,Bercy,1,1946-1970,meublé,28.8,34.6,20.2,7511247,48.835209,2.386210
2,2025,Chaussée-d'Antin,2,1946-1970,non meublé,28.5,34.2,20.0,7510934,48.873547,2.332269
3,2025,Notre-Dame-des-Champs,3,Apres 1990,non meublé,30.2,36.2,21.1,7510623,48.846428,2.327357
4,2023,Muette,4,Apres 1990,meublé,29.3,35.2,20.5,7511662,48.863275,2.259936


Création d'un CSV de notre base nettoyé

In [ ]:
df.to_csv("loyers_nettoye.csv", sep=';', index=False, encoding='utf-8')